In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
import re

爬取租房数据

In [9]:
service = Service(r'D:\anaconda3\chromedriver.exe') 
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://sh.zu.fang.com/')

In [10]:
link_1 = driver.find_element(By.LINK_TEXT,'浦东')
link_1.click()
sleep(0.3)
link_2 = driver.find_element(By.LINK_TEXT,'世博')
link_2.click()
sleep(0.3)

In [11]:
page = driver.find_element(By.XPATH,'//*[@id="rentid_D10_01"]/span')
page_number = min(int(re.search(r"共(\d+)页", page.text).group(1)),20)

all_page_information = []
count = 0

while count < page_number:

    table = driver.find_element(By.CLASS_NAME,'houseList')
    rows = table.find_elements(By.CLASS_NAME,'list.hiddenMap.rel') # Direct child element
    House_discription = []
    House_information = []
    House_location = []
    House_distance_subway = []
    House_characteristic = []
    House_price = []
    for row in rows:
        try:
            discription = row.find_element(By.CLASS_NAME,'title')
            House_discription.append(discription.text)
        except:
            House_discription.append(None)

        try:
            information = row.find_element(By.CLASS_NAME,'font15.mt12.bold')
            House_information.append(information.text)
        except:
            House_information.append(None)

        try:
            location = row.find_element(By.CLASS_NAME,'gray6.mt12')
            House_location.append(location.text)
        except:
            House_location.append(None)

        try:
            distance_subway = row.find_element(By.CLASS_NAME,'note.subInfor')
            House_distance_subway.append(distance_subway.text)
        except:
            House_distance_subway.append(None)

        try:
            characteristic_1 = row.find_element(By.CLASS_NAME,'note.colorGreen')
            characteristic_2 = row.find_element(By.CLASS_NAME,'note.colorRed')
            characteristic_3 = row.find_element(By.CLASS_NAME,'note.colorBlue')
            characteristic = [characteristic_1.text,characteristic_2.text,characteristic_3.text]
            House_characteristic.append(characteristic)
        except:
            House_characteristic.append([None,None,None])
        
        try:
            price = row.find_element(By.CLASS_NAME,'price')
            House_price.append(price.text)
        except:
            House_price.append(None)

    df_discription = pd.DataFrame(House_discription,columns=['House_discription'])
    df_information = pd.DataFrame(House_information,columns=['House_information'])
    df_location = pd.DataFrame(House_location,columns=['House_location'])
    df_distance_subway = pd.DataFrame(House_distance_subway,columns=['House_distance_subway'])
    df_House_Characteristic = pd.DataFrame(House_characteristic,columns=['House_characteristic_1','House_characteristic_2','House_characteristic_3'])
    df_price = pd.DataFrame(House_price,columns=['House_price'])
    all_information = pd.concat([df_discription,df_information,df_location,df_distance_subway,df_House_Characteristic,df_price],axis=1)

    information_split = []
    for s in df_information['House_information']:
        try:
            split = s.split('|')
            
            room_hall = split[1]
            room_match = re.match(r"(\d+)室(\d+)厅",room_hall)
            bed_room_number = int(room_match.group(1))
            living_room_number = int(room_match.group(2))

            room_area = split[2]
            room_area_match = re.search(r'(\d+\.\d+|\d+)',room_area)
            room_area_number = float(room_area_match.group())
            
            info_group = [split[0],bed_room_number,living_room_number,room_area_number,split[3]]
            information_split.append(info_group)
        except:
            information_split.append([None,None,None,None,None])
    df_information_split = pd.DataFrame(information_split)


    location_split = []
    for s in df_location['House_location']:
        try:
            split = s.split('-')
            location_split.append(split)
        except:
            location_split.append([None,None,None])
    df_location_split = pd.DataFrame(location_split)

    distance_subway = []
    for s in df_distance_subway['House_distance_subway']:
        try:
            distance = int(re.search(r'约(\d+)', s).group(1))
            distance_subway.append(distance)
        except:
            distance_subway.append(None)
    df_distance_subway_split = pd.DataFrame(distance_subway)

    one_page_information = pd.concat([df_discription,df_information_split,df_location_split,df_distance_subway_split,df_House_Characteristic,df_price],axis=1)
    all_page_information.append(one_page_information)

    try:
        turn_page = driver.find_element(By.LINK_TEXT,'下一页')
        turn_page.click()
        sleep(0.5)
        count = count+1
    except:
        break
    

df_all_page_information = pd.concat(all_page_information,axis=0)
new_columns = ['描述','租赁方式','室数','厅数','面积','朝向','一级地址','二级地址','三级地址','距离地铁站距离','特征1','特征2','特征3','价格']
df_all_page_information.columns = new_columns
df_all_page_information.to_excel('rent_house.xlsx',index=False)



